In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow_probability as tfp
# from google.colab import drive
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
import PIL.Image
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [2]:
tfpl = tfp.layers
tfd = tfp.distributions

In [3]:
data_dir = 'formatted_split'
img_height, img_width = 256, 256
batch_size = 64

from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Convert string labels to integers
le = LabelEncoder()

def custom_data_generator(file_paths, labels, batch_size):
    num_samples = len(file_paths)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i : min(i + batch_size, num_samples)]
            batch_file_paths = [file_paths[j] for j in batch_indices]
            batch_labels = [labels[j] for j in batch_indices]

            # Load and preprocess the batch of images
            batch_images = [img_to_array(load_img(file, target_size=(img_height, img_width))) for file in batch_file_paths]
            batch_images = np.array(batch_images) / 255.0  # Properly rescale to [0, 1]
            batch_labels = le.transform(batch_labels)
            batch_labels = to_categorical(batch_labels, num_classes=num_classes)  # One-hot encode the labels

            yield np.array(batch_images), np.array(batch_labels)

# Create data generators
train_generator = custom_data_generator(train_files, train_labels, batch_size)
validation_generator = custom_data_generator(val_files, val_labels, batch_size)
test_generator = custom_data_generator(test_files, test_labels, batch_size)


NameError: name 'train_files' is not defined

In [ ]:

num_train_images = len(train_files)
num_validation_images = len(val_files)
num_test_images = len(test_files)

print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_validation_images}")
print(f"Number of test images: {num_test_images}")


In [5]:
# Divergence function for Bayesian layers
divergence_fn = lambda q, p, _: tfd.kl_divergence(q, p) / num_train_images

# Build the Bayesian CNN model
model_bayes = Sequential([
    tfpl.Convolution2DReparameterization(input_shape=(255,255, 3),
                                          filters=8, kernel_size=16, activation='relu',
                                          kernel_prior_fn=tfpl.default_multivariate_normal_fn,
                                          kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                          kernel_divergence_fn=divergence_fn,
                                          bias_prior_fn=tfpl.default_multivariate_normal_fn,
                                          bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                          bias_divergence_fn=divergence_fn),
    Conv2D(32, (3,3), activation='relu'),
     MaxPooling2D(2,2),
    Dropout(0.4),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.4),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.4),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Dropout(0.3),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    tfpl.DenseReparameterization(units=tfpl.OneHotCategorical.params_size(2), activation=None,
                                 kernel_prior_fn=tfpl.default_multivariate_normal_fn,
                                 kernel_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                 kernel_divergence_fn=divergence_fn,
                                 bias_prior_fn=tfpl.default_multivariate_normal_fn,
                                 bias_posterior_fn=tfpl.default_mean_field_normal_fn(is_singular=False),
                                 bias_divergence_fn=divergence_fn
                                ),
    tfpl.OneHotCategorical(2)
])



model_bayes.summary()



C:\Users\Public\anaconda\Lib\site-packages\tensorflow_probability\python\layers\util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
C:\Users\Public\anaconda\Lib\site-packages\tensorflow_probability\python\layers\util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


NameError: Exception encountered when calling layer "conv2d_reparameterization" (type Conv2DReparameterization).

name 'num_train_images' is not defined

Call arguments received by layer "conv2d_reparameterization" (type Conv2DReparameterization):
  • inputs=tf.Tensor(shape=(None, 255, 255, 3), dtype=float32)

In [6]:
# Define a custom negative log likelihood loss function
def negative_log_likelihood(y_true, y_pred):
    # Extract the parameters of the predicted distribution
    y_true = tf.cast(y_true, tf.float32)

    return -y_pred.log_prob(y_true)

    # Calculate the negative log likelihood of the true labels
    neg_log_likelihood = -tf.reduce_sum(y_true * logits - tf.math.log(tf.reduce_sum(tf.exp(logits), axis=-1)), axis=-1)

    return tf.reduce_mean(neg_log_likelihood)

# Compile the model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model_bayes.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0025),
                    loss=negative_log_likelihood,
                    metrics=['accuracy'])

epochs = 15  

history = model_bayes.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_images // batch_size,
    callbacks=[reduce_lr],
    verbose=1
)

# Evaluate the model on the validation set
val_loss, val_accuracy = model_bayes.evaluate(validation_generator, verbose=1)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}")


NameError: name 'model_bayes' is not defined

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))

# Plot training & validation accuracy values
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import  cv2

# 1. Probability Plot for a Given Image

def plot_image_probabilities(model, image_path):
    # Load and preprocess the image
    img_height, img_width = 255, 255  # Define the target size

    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))

    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the image
    predictions = model(img_array)

    # Extract probabilities
    probs = predictions.mean().numpy()[0]

    # Plot
    plt.figure(figsize=(8, 4))
    plt.bar([0, 1], probs, color=['blue', 'red'])
    plt.xticks([0, 1], ['Not Infected', 'Infected'])
    plt.ylabel('Probability')
    plt.title('Prediction Probabilities')
    plt.show()

# Testing with an image
image_path = './Image_1.jpg'  
plot_image_probabilities(model_bayes, image_path)

image_path = './Image_2.jpg'  
plot_image_probabilities(model_bayes, image_path)

image_path = './Image_3.jpg'  
plot_image_probabilities(model_bayes, image_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import  cv2

# 1. Probability Plot for a Given Image

def plot_image_probabilities(model, image_path):
    # Load and preprocess the image
    img_height, img_width = 255, 255  # Define the target size

    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))

    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict the image
    predictions = model(img_array)

    # Extract probabilities
    probs = predictions.mean().numpy()[0]

    # Plot
    plt.figure(figsize=(8, 4))
    plt.bar([0, 1], probs, color=['pink', 'grey'])
    plt.xticks([0, 1], ['Normal', 'Infected'])
    plt.ylabel('Probability')
    plt.title('Prediction Probabilities')
    plt.show()

# Testing with an image
image_path = 'real_time_pre/normal_4.jpg'  
plot_image_probabilities(model_bayes, image_path)

#image_path = './Image_2.jpg'  
#plot_image_probabilities(model_bayes, image_path)

#image_path = './Image_3.jpg'  
#plot_image_probabilities(model_bayes, image_path)